In [1]:
from pathlib import Path
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import soundfile as sf
import pandas as pd

import os 
import itertools
import re
import math
import csv
import random
from sklearn import tree
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from scipy.fftpack import fftn, ifftn, fft, ifft
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy import signal
import pickle
from sklearn.externals import joblib
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
import warnings

warnings.filterwarnings('ignore') 
df = pd.read_csv("noAudible/Features_594too599hz.csv")



#df = pd.read_csv("Features_all_no_audible.csv")
#df = df[(df.modeID == 'whisper')]
df

C:\Users\Magomed\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


channel_1_mav  channel_1_mav2  channel_1_rms  channel_1_iav  \
0        103.905497       68.332739     221.012500   4.059588e+05   
1          0.060816        0.039893       0.103650   1.598849e+02   
2        593.577932      403.033115    1354.512639   2.809404e+06   
3      27539.639597    17472.705337   68709.486312   1.338151e+08   
4      69565.827602    50166.055351  156621.532472   3.814990e+08   
...             ...             ...            ...            ...   
4795       0.307272        0.205215       0.540942   8.373158e+02   
4796       0.016279        0.010399       0.026301   3.617217e+01   
4797       0.018088        0.012766       0.027480   4.755301e+01   
4798       0.012599        0.009039       0.018352   2.515937e+01   
4799       0.089150        0.060745       0.143031   2.037080e+02   

      channel_1_ssi  channel_1_var  channel_1_wl  channel_1_iemg  \
0      1.908434e+08   4.884651e+04  8.109185e+05    4.059588e+05   
1      2.824403e+01   1.074326e-02  3.194363e+02    1.598849e+02   
2      8.683656e+09   1.834704e+06  5.612663e+06    2.809404e+06   
3      2.293931e+13   4.720992e+09  2.672744e+08    1.338151e+08   
4      1.345242e+14   2.453030e+10  7.622981e+08    3.814990e+08   
...             ...            ...           ...             ...   
4795   7.973848e+02   2.926181e-01  1.672651e+03    8.373158e+02   
4796   1.537073e+00   6.917517e-04  7.226184e+01    3.617217e+01   
4797   1.985337e+00   7.551678e-04  9.501570e+01    4.755301e+01   
4798   6.725778e-01   3.367939e-04  5.026141e+01    2.515937e+01   
4799   4.674647e+01   2.045796e-02  4.068995e+02    2.037080e+02   

      channel_1_aac  channel_1_zc  ...  channel_6_aac  channel_6_zc  \
0        207.555275          3903  ...     430.253705          3903   
1          0.121505          2624  ...       1.010074          2625   
2       1185.857287          4730  ...    3689.232947          4729   
3      55006.049146          4855  ...    1718.349703          4855   
4     139004.033793          5479  ...    2053.865999          5479   
...             ...           ...  ...            ...           ...   
4795       0.613817          2721  ...       0.005753          2710   
4796       0.032521          2219  ...       0.016909          2217   
4797       0.036141          2624  ...       0.015562          2621   
4798       0.025168          1992  ...       0.009605          1992   
4799       0.178074          2280  ...       0.023294          2280   

      channel_6_ssc  channel_6_wamp  channel_6_medf  channel_6_meanf  userID  \
0              3902            1902       10.796915        45.455760       1   
1              2624             675       10.344828        44.885212       1   
2              4728            2335       12.076271        46.819841       1   
3              4854            2420       11.157025        45.255710       1   
4              5478            2658       11.517367        45.976100       1   
...             ...             ...             ...              ...     ...   
4795           2709               0        8.856089        24.867733       8   
4796           2216               0        9.502262        42.710357       8   
4797           2620               0       10.344828        42.792458       8   
4798           1991               0        7.575758        41.304095       8   
4799           2279               0       10.572687        44.170220       8   

      sessionID     uttID   modeID  
0          1001  10010001  audible  
1          1001  10010002  audible  
2          1001  10010003  audible  
3          1001  10010004  audible  
4          1001  10010005  audible  
...         ...       ...      ...  
4795       8019  80190246  whisper  
4796       8019  80190247  whisper  
4797       8019  80190248  whisper  
4798       8019  80190249  whisper  
4799       8019  80190250  whisper  

[4800 rows x 88 columns]

In [2]:
def split_data_per_session(session_id):
    temp_df = df[(df.sessionID == session_id)]
    data = temp_df.copy()
    data = shuffle(data)
    y = data['modeID']
    data.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return train_test_split(data, y, test_size=0.2)

def split_data_session(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['sessionID'].isin(train_user_ids)]
    test_df = data.loc[data['sessionID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

def split_data_utterance(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['userID'].isin(train_user_ids)]
    test_df = data.loc[data['userID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

def split_data(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['userID'].isin(train_user_ids)]
    test_df = data.loc[data['userID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

In [3]:
def lda_user1():
    train_ids = [2,3,4,5,6,7,8]
    test_ids = [1]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score     
def lda_user2():
    train_ids = [1,3,4,5,6,7,8]
    test_ids = [2]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score  
    
def lda_user3():
    train_ids = [1,2,4,5,6,7,8]
    test_ids = [3]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score 
    
def lda_user4():
    train_ids = [1,2,3,5,6,7,8]
    test_ids = [4]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score     
        
def lda_user5():
    train_ids = [1,2,3,4,6,7,8]
    test_ids = [5]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score   
    
def lda_user6():
    train_ids = [1,2,3,4,5,7,8]
    test_ids = [6]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score  
    
def lda_user7():
    train_ids = [1,2,3,4,5,6,8]
    test_ids = [7]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score    
    
def lda_user8():
    train_ids = [1,2,3,4,5,6,7]
    test_ids = [8]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score 

In [4]:
def lda_users():
    a = lda_user1()
    b = lda_user2()
    c = lda_user3()
    d = lda_user4()
    e = lda_user5()
    f = lda_user6()
    g = lda_user7()
    h = lda_user8()
    scores = np.array([a,b,c,d,e,f,g,h])
    print("Durchschnitt: ")
    print (scores.mean())
    print ("Standartabweichung: ")
    print (scores.std())

In [5]:
lda_users()

Accuracy for session 19 is 0.35555555555555557
Accuracy for session 19 is 0.33111111111111113
Accuracy for session 19 is 0.34
Accuracy for session 19 is 0.28
Accuracy for session 19 is 0.31333333333333335
Accuracy for session 19 is 0.3466666666666667
Accuracy for session 19 is 0.3
Accuracy for session 19 is 0.3028571428571429
Durchschnitt: 
0.32119047619047625
Standartabweichung: 
0.024560510884046396
